## สคลิปทดสอบนี้ ใช้สำหรับ ทดสอบผ่านข้อความให้กับ LLM แล้ว
## ใช้คำถาม ถามตามข้อความที่ส่งให้

In [1]:
from constants import *
from custom_gpt4all import pyllmodel
from langchain.prompts import PromptTemplate

model_path = f'{LLM_MODEL_PATH}\\{LLM_MODEL_NAME}'
llm = pyllmodel.LLModel()
llm.load_model(model_path=model_path)

d:\Program Files\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [24]:
query_str = """
date� *** Payment date� of cheque is the date that specify on the cheque *** �Clearing date� of cheque is the �payment date� + 1 official day Details of deposition, if customers pay by cheque or cash,Cheque T/R (or company cheuqe) : the system will increase customer cash balance on the �payment date�, customer can withdraw this amount after �payment date� *** Payment date� of cheque is the date  \n\n\nQuestion: What is the payment date?
"""
# 

llm.generate(
     prompt=query_str,
        streaming=True,
        temp=0.8,

        # จำนวน Token (ชุดของข้อความที่มีความเป็นไปได้ ที่ใกล้งเคียงกับคำถาม) ที่บอกให้ LLM หยิบมาใช้ในการสร้างประโยคคำตอบ
        # ซึ่งรายการ Token ที่ได้จะถูกจัดลำดับ (sort) ตามความน่าจะเป็นที่ มากสุดไปน้อย สุด
        top_k=20, 


        # ผลรวม % จากจำนวนทั้งหมดของ top_k (คือ 20 รายการ)
        # จะต้องได้ >= top_p ถึงจะออกมาเป็นคำตอบ
        # หลักๆ แล้วจะใช้ในการ กรองข้อมูล Token ที่มีความน่าจะเป็น ของคำตอบน้อย ออกไป
        top_p=0.75,

        # จำนวนคำ (Token) สูงสุด ที่นำมาสร้างเป็นรูปประโยคคำตอบ สูงสุดไม่เกินจำนวนกี่คำ
        n_predict = 128,


        # จำนวนคำทำนาย หรือ การคาดการณ์ หรือ คำถาม ที่จะใช้ถาม เพื่อให้ได้คำตอบ
        # กรณีในคำตอบ ที่ได้นั้น มีคำตอบ อยู่ภายในรูปแบบประโยคคำตอบ
        # ถ้ากำหนดค่า n_bath 2,3,4,5, ... LLM ก็จะสร้างคำตอบเพิ่มเติม ตามจำนวนของ n_batch และ จำนวนของคำถามที่พบในคำตอบ
        n_batch = 1,

        #repeat_last_n = 15,
        #repeat_penalty=2.7,
)

Answer: No. It depends on the Share Status as it is not always available. Explanation: The Buy and Sell options are available when the Share Status is Active, but it can vary depending on the Account Status.


'Answer: No. It depends on the Share Status as it is not always available. Explanation: The Buy and Sell options are available when the Share Status is Active, but it can vary depending on the Account Status.'